In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


In [5]:
df=pd.read_csv("~/Git/parser/lyrics.csv")
stopwords = stopwords.words("english")
# stopwords.extend([])

In [6]:
raw = df.iloc[0:107,0]


In [8]:
p_stemmer = nltk.PorterStemmer()
tokenizer = nltk.RegexpTokenizer(r'\w+')

texts_list=[]

for t in raw:
    tokens = tokenizer.tokenize(t.lower()) #split to 1 word tokens
    stopped_tokens = [i for i in tokens if not i in stopwords] #check for stopwords
    text = [p_stemmer.stem(i) for i in stopped_tokens] #lemmatization or smth
    texts_list.append(text)


In [16]:
bigram = gensim.models.Phrases(texts_list, min_count=6, threshold=120) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[texts_list], threshold=120)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


In [17]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [18]:
data_words_bigrams = make_bigrams(texts_list)
data_words_bigrams_trigrams = make_trigrams(data_words_bigrams)

In [19]:
#tf-idf
from gensim.models import TfidfModel

dictionary = corpora.Dictionary(data_words_bigrams_trigrams)
corpus = [dictionary.doc2bow(text) for text in data_words_bigrams_trigrams]

tfidf= TfidfModel(corpus, id2word = dictionary)

low_value = 0.3
low_value_words = []
for bow in corpus:
    low_value_words += [id for id, value in tfidf[bow] if value < low_value]

dictionary.filter_tokens(bad_ids=low_value_words)

new_corpus = [dictionary.doc2bow(text) for text in data_words_bigrams_trigrams]


In [29]:
ldamodel = gensim.models.ldamodel.LdaModel(new_corpus, num_topics=6, id2word = dictionary, passes=60)

In [30]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel,new_corpus,dictionary)

/home/gselivirov/anaconda3/envs/nlp/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [31]:
pyLDAvis.display(vis)